# 08 Design pattern and checklist
Hypothesis: recommended methods should balance accuracy and stability, not only maximize endpoint score.

### Expected Outcome
A robust recommendation should stay near the accuracy-stability frontier with bounded drift.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.notebook_transfer import TransferNotebookLab

LAB = TransferNotebookLab.from_root(ROOT)
FAST_DEV_RUN = False

In [ ]:
methods = ['scratch', 'feature_extraction', 'gradual_unfreeze', 'naive_finetune']
LAB.run('transfer_core_related.yaml', '08_checklist', fast_dev_run=FAST_DEV_RUN)
frames = LAB.load_methods(methods)
summary = pd.DataFrame([
    {
        'method': m,
        'final_target_acc': float(df['target_test_acc'].iloc[-1]),
        'final_feature_drift': float(df['feature_drift'].iloc[-1]),
        'final_retention': float(df['source_retention_acc'].iloc[-1]) if 'source_retention_acc' in df else float('nan'),
        'final_grad_norm': float(df['grad_norm'].iloc[-1]),
    }
    for m, df in frames.items()
]).sort_values('final_target_acc', ascending=False)

stable = summary[summary['final_feature_drift'] <= 0.25]
recommended = stable.sort_values('final_target_acc', ascending=False).iloc[0]['method'] if not stable.empty else summary.iloc[0]['method']
summary

In [ ]:
fig, ax = plt.subplots(figsize=(6.7, 3.7))
for m, df in frames.items():
    ax.plot(df['epoch'], df['target_test_acc'], marker='o', label=m)
ax.set_title('Target accuracy overview')
ax.set_xlabel('epoch')
ax.set_ylabel('target_test_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '08_target_accuracy_overview.png')

fig, ax = plt.subplots(figsize=(6.2, 3.8))
for _, row in summary.iterrows():
    edge = 'black' if row['method'] == recommended else 'none'
    size = 90 if row['method'] == recommended else 55
    ax.scatter(row['final_feature_drift'], row['final_target_acc'], s=size, edgecolors=edge)
    ax.text(row['final_feature_drift'] + 0.004, row['final_target_acc'] + 0.002, row['method'], fontsize=9)
ax.set_title('Accuracy-stability frontier')
ax.set_xlabel('final_feature_drift (lower is better)')
ax.set_ylabel('final_target_acc')
ax.grid(alpha=0.25)
LAB.savefig(fig, '08_accuracy_stability_frontier.png')

In [ ]:
f'Recommended default: {recommended}'

### Final Checklist
1. Validate task relatedness first.
2. Prefer low-drift methods with competitive target accuracy.
3. Compare against scratch before claiming transfer benefit.